<a href="https://colab.research.google.com/github/hakloi/chatbotik/blob/main/chatbot_airexpress_faq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [40]:
import re
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
import pandas as pd
import zipfile
import os

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"violettapatrusheva","key":"735b582d56d8af4eb8a4a15e04bb9d2e"}'}

In [43]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [44]:
!pip install -q kaggle

In [45]:
!kaggle datasets download -d abbbhishekkk/faq-datasets-for-chatbot-training


Dataset URL: https://www.kaggle.com/datasets/abbbhishekkk/faq-datasets-for-chatbot-training
License(s): unknown
faq-datasets-for-chatbot-training.zip: Skipping, found more recently modified local copy (use --force to force download)


In [46]:
with zipfile.ZipFile("faq-datasets-for-chatbot-training.zip", 'r') as zip_ref:
    zip_ref.extractall("faq_data")

In [47]:
os.listdir("faq_data")

['faq_results.txt',
 'Tata_comm_faq.txt',
 'Amazon_sagemaker_Faq.txt',
 'Aadhar_Faq.txt',
 'Sevenhillshospital_faq.txt',
 'HDFC_Faq.txt']

In [48]:
df = pd.read_json("faq_data/faq_results.txt")
df.head()

,Question,Answer
0,How do I book my flight?,You can make a new booking by any of the follo...
1,What is the maximum number of seats that I can...,You can book up to nine passengers in a single...
2,How can I book a seat for my infant?,Booking an infant seat can be done through our...
3,My ticket was purchased from a travel agent. C...,"Yes, You can modify your booking under Manage ..."
4,How can I pay for my ticket online?,"You can use a credit card, debit card or Net B..."


In [49]:
# # Test

# query = "hello neighbour!"
# tokens = word_tokenize(query)
# tokens

In [50]:
def clean_txt(txt):
  text = txt.lower()
  text = re.sub(r'\s+', ' ', text)  # убираем лишние пробелы
  text = text.strip()
  words = word_tokenize(txt)
  words = [word for word in words if word not in stop_words]
  return " ".join(words)

def remove_question_mark(text):
    return text.translate(str.maketrans('', '', string.punctuation))


In [58]:
def get_answer(q):
  cleaned = clean_txt(q)
  user_vect = vectorizer.transform([cleaned])
  # Сравнение с обученной матрицей
  similarities = cosine_similarity(user_vect, tfidf_matrix)
  best_match = similarities.argmax()

  return df.iloc[best_match]['Answer']

In [51]:
df['cleaned_question'] = df['Question'].apply(lambda x: remove_question_mark(clean_txt(x)))
df['cleaned_answer'] = df['Answer'].apply(lambda x: remove_question_mark(clean_txt(x)))

df.head()

,Question,Answer,cleaned_question,cleaned_answer
0,How do I book my flight?,You can make a new booking by any of the follo...,How I book flight,You make new booking following methods Buy sea...
1,What is the maximum number of seats that I can...,You can book up to nine passengers in a single...,What maximum number seats I book time,You book nine passengers single booking inclu...
2,How can I book a seat for my infant?,Booking an infant seat can be done through our...,How I book seat infant,Booking infant seat done contact centre Air In...
3,My ticket was purchased from a travel agent. C...,"Yes, You can modify your booking under Manage ...",My ticket purchased travel agent Can I change...,Yes You modify booking Manage Booking section...
4,How can I pay for my ticket online?,"You can use a credit card, debit card or Net B...",How I pay ticket online,You use credit card debit card Net Banking O...


In [52]:
# test

print("Before cleaning:", df['Question'][0])
print("After cleaning:", df['cleaned_question'][0])

Before cleaning: How do I book my flight?
After cleaning: How I book flight 


In [54]:
# векторизуем очищенные вопросы через TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned_question'])

In [55]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 323 stored elements and shape (48, 153)>

In [59]:
# test
user_q = "how to book seat for infant?"
print(get_answer(user_q))

Booking an infant seat can be done through our contact centre or Air India Express overseas offices by requesting to make the booking under child category.


In [64]:
def chat():
  while True:
    user = input("You:")
    if user.lower() in r"bye|goodbye|bb|exit|quit":
      print("Have a good day!")
      break
    else:
      ans = get_answer(user)
      print("Helper: " + ans)

In [65]:
chat()

You:Hi
Helper: You can make a new booking by any of the following methods Buy a seat Online at www.airindiaexpress.in Walk into any of Air India Express' airport or city offices to purchase over the counter. Visit one of our approved Travel Agents. You can also book through our 24x7 Call Centre @ 0091-44-40013001
You:infant
Helper: The same procedure has to be applied as is applicable for booking an adult. Infant must be booked with the accompanying adult. The date of birth has to be filled in while making the booking.
You:mel
Helper: You can make a new booking by any of the following methods Buy a seat Online at www.airindiaexpress.in Walk into any of Air India Express' airport or city offices to purchase over the counter. Visit one of our approved Travel Agents. You can also book through our 24x7 Call Centre @ 0091-44-40013001
You:meal
Helper: You can pre-book your on-board meal 24 hours prior to the departure of your flight.
You:drinks ?
Helper: Air India Express offers pre-set meal